# NB 6.1: Entity Extraction

## Part 0: Importing Libraries

In [1]:
import os
import requests
import re
import sys
import regex as re

In [2]:
import spacy

# Using Spacy Large
# !python -m spacy download en_core_web_lg
nlp = spacy.load("en_core_web_lg", disable=["tagger", "parser"])

# Enable only NER
nlp.enable_pipe("ner")
nlp

2023-05-23 00:00:09.606605: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-23 00:00:10.792332: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2023-05-23 00:00:10.792497: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/loca

In [3]:
import nltk as nltk
import nltk.corpus  
from nltk.text import Text

# nltk.download('punkt')
# nltk.download('stopwords')
# nltk.download('wordnet')

In [2]:
import pandas as pd
import numpy as np
import sklearn

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
# Graphics in SVG format are more sharp and legible
%config InlineBackend.figure_format = 'svg'
import warnings
warnings.filterwarnings("ignore")

In [3]:
#pip install pandarallel
import multiprocessing

num_processors = multiprocessing.cpu_count()
print(f'Available CPUs: {num_processors}')

import pandarallel
from pandarallel import pandarallel
pandarallel.initialize(nb_workers=num_processors-1, use_memory_fs=False, progress_bar=True)

Available CPUs: 16
INFO: Pandarallel will run on 15 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [4]:
# Imports the Google Cloud client library
from google.cloud import storage
# Instantiates a client
storage_client = storage.Client()

# The name for the new bucket
bucket_name = "nlp_final_project_kshitijm"

# Creates the new bucket
bucket = storage_client.bucket(bucket_name)
print(f"Bucket {bucket.name} connected.")

Bucket nlp_final_project_kshitijm connected.


In [5]:
from tqdm import tqdm
from collections import Counter
from itertools import chain

## Part 1: Running NER for each year

In [8]:
def spacy_ner(x):
  doc = nlp(x)
  entities = []
  labels = []
  for ent in doc.ents:
    entities.append(ent.text)
    labels.append(ent.label_)
  entities_labels = list(zip(entities, labels)) #we do not want unique entities
  return entities_labels

**2020**

In [8]:
%%time
df_filt_2020=pd.read_csv('gs://nlp_final_project_kshitijm/00_Data/NLP_FP_Data5_2020_Topics_Sentiments.csv',lineterminator='\n')
df_filt_2020.head(3)

CPU times: user 19 s, sys: 2.51 s, total: 21.5 s
Wall time: 1min 1s


,Unnamed: 0,url,date,language,title,text,cleaned_text,article_source,clean_title,title_tokens,...,year_pub,month,month-year,flag_relevant,num_tokens,rake_phrases_articles,rake_phrases_joined,final_topic,sent_probs,sent_label
0,0,https://fusionscienceacademy.com/artificial-in...,2020-01-30,en,Artificial Intelligence (AI) in Social Media ...,\n\nArtificial Intelligence (AI) in Social Med...,Artificial Intelligence AI in Social MediaMar...,2026 – Fusion Science Academy,Artificial Intelligence (AI) in Social Media ...,"['artificial', 'intelligence', 'ai', 'social',...",...,2020,1,Jan 2020,1,22283,['burkert fluid control systems emerson electr...,burkert fluid control systems emerson electric...,6,[[2.1574432e-04 9.9863547e-01 1.1488239e-03]],1
1,1,https://health.economictimes.indiatimes.com/ne...,2020-01-10,en,artificial intelligence: Researchers develop A...,\n\nartificial intelligence: Researchers devel...,artificial intelligence: Researchers develop ...,NaN,artificial intelligence: Researchers develop A...,"['artificial', 'intelligence', 'researchers', ...",...,2020,1,Jan 2020,1,8087,['economic times ethealthworldhome news hospit...,economic times ethealthworldhome news hospital...,1,[[1.9508268e-04 8.0524624e-04 9.9899966e-01]],2
2,2,https://heraldpublicist.com/bet-gil-on-ai-fina...,2020-01-15,en,Bet Gil on AI Final Fantasy Tactics Matches in...,\n\nBet Gil on AI Final Fantasy Tactics Matche...,Bet Gil on AI Final Fantasy Tactics Matches i...,Herald Publicist,Bet Gil on AI Final Fantasy Tactics Matches in...,"['bet', 'gil', 'ai', 'final', 'fantasy', 'tact...",...,2020,1,Jan 2020,1,4458,['hilarious twitch streamnewstechnologycricket...,hilarious twitch streamnewstechnologycricketpo...,1,[[5.5355614e-04 9.9602497e-01 3.4214482e-03]],1


In [11]:
%%time
tqdm.pandas()
df_filt_2020['entities_spacy'] = df_filt_2020['cleaned_text'].parallel_apply(lambda x: spacy_ner(x)).progress_apply(lambda x: x)

100%|██████████| 32667/32667 [00:00<00:00, 786346.25it/s]

CPU times: user 35.9 s, sys: 4.79 s, total: 40.7 s
Wall time: 8min 14s


In [ ]:
df_filt_2020['ENT_ORG'] = df_filt_2020['entities_spacy'].parallel_apply(lambda x: [tup[0] for tup in x if tup[1] == "ORG"])
df_filt_2020['ENT_PROD'] = df_filt_2020['entities_spacy'].parallel_apply(lambda x: [tup[0] for tup in x if tup[1] == "PRODUCT"])
df_filt_2020['ENT_PER'] = df_filt_2020['entities_spacy'].parallel_apply(lambda x: [tup[0] for tup in x if tup[1] == "PERSON"])
df_filt_2020['ENT_NORP'] = df_filt_2020['entities_spacy'].parallel_apply(lambda x: [tup[0] for tup in x if tup[1] == "NORP"])

In [14]:
%%time

##SAVED AS THE WRONG FILE NAME BY MISTAKE, CORRECT LATER
df_filt_2020.to_csv('gs://nlp_final_project_kshitijm/00_Data/NLP_FP_Data5_2021_Topics_Sentiments_NER.csv')

CPU times: user 35.5 s, sys: 525 ms, total: 36 s
Wall time: 1min 4s


----
**2021**

In [24]:
df_filt_2021=pd.read_csv('gs://nlp_final_project_kshitijm/00_Data/NLP_FP_Data5_2021_Topics_Sentiments.csv',lineterminator='\n')
df_filt_2021['entities_spacy'] = df_filt_2021['cleaned_text'].parallel_apply(lambda x: spacy_ner(x))

In [ ]:
df_filt_2021['ENT_ORG'] = df_filt_2021['entities_spacy'].parallel_apply(lambda x: [tup[0] for tup in x if tup[1] == "ORG"])
df_filt_2021['ENT_PROD'] = df_filt_2021['entities_spacy'].parallel_apply(lambda x: [tup[0] for tup in x if tup[1] == "PRODUCT"])
df_filt_2021['ENT_PER'] = df_filt_2021['entities_spacy'].parallel_apply(lambda x: [tup[0] for tup in x if tup[1] == "PERSON"])
df_filt_2021['ENT_NORP'] = df_filt_2021['entities_spacy'].parallel_apply(lambda x: [tup[0] for tup in x if tup[1] == "NORP"])

In [26]:
%%time
df_filt_2021.to_csv('gs://nlp_final_project_kshitijm/00_Data/NLP_FP_Data6_2021v2_Topics_Sentiments_NER.csv')

CPU times: user 46.5 s, sys: 808 ms, total: 47.3 s
Wall time: 1min 18s


------
**2022**

In [10]:
%%time
df_filt_2022=pd.read_csv('gs://nlp_final_project_kshitijm/00_Data/NLP_FP_Data5_2022_Topics_Sentiments.csv',lineterminator='\n')
df_filt_2022['entities_spacy'] = df_filt_2022['cleaned_text'].parallel_apply(lambda x: spacy_ner(x))

df_filt_2022['ENT_ORG'] = df_filt_2022['entities_spacy'].parallel_apply(lambda x: [tup[0] for tup in x if tup[1] == "ORG"])
df_filt_2022['ENT_PROD'] = df_filt_2022['entities_spacy'].parallel_apply(lambda x: [tup[0] for tup in x if tup[1] == "PRODUCT"])
df_filt_2022['ENT_PER'] = df_filt_2022['entities_spacy'].parallel_apply(lambda x: [tup[0] for tup in x if tup[1] == "PERSON"])
df_filt_2022['ENT_NORP'] = df_filt_2022['entities_spacy'].parallel_apply(lambda x: [tup[0] for tup in x if tup[1] == "NORP"])

df_filt_2022.to_csv('gs://nlp_final_project_kshitijm/00_Data/NLP_FP_Data6_2022_Topics_Sentiments_NER.csv', index=False)

CPU times: user 2min 28s, sys: 19.7 s, total: 2min 48s
Wall time: 16min 26s


-----
**2023**

In [11]:
%%time
df_filt_2023=pd.read_csv('gs://nlp_final_project_kshitijm/00_Data/NLP_FP_Data5_2023_Topics_Sentiments.csv',lineterminator='\n')
df_filt_2023['entities_spacy'] = df_filt_2023['cleaned_text'].parallel_apply(lambda x: spacy_ner(x))

df_filt_2023['ENT_ORG'] = df_filt_2023['entities_spacy'].parallel_apply(lambda x: [tup[0] for tup in x if tup[1] == "ORG"])
df_filt_2023['ENT_PROD'] = df_filt_2023['entities_spacy'].parallel_apply(lambda x: [tup[0] for tup in x if tup[1] == "PRODUCT"])
df_filt_2023['ENT_PER'] = df_filt_2023['entities_spacy'].parallel_apply(lambda x: [tup[0] for tup in x if tup[1] == "PERSON"])
df_filt_2023['ENT_NORP'] = df_filt_2023['entities_spacy'].parallel_apply(lambda x: [tup[0] for tup in x if tup[1] == "NORP"])

df_filt_2023.to_csv('gs://nlp_final_project_kshitijm/00_Data/NLP_FP_Data6_2023_Topics_Sentiments_NER.csv', index=False)

CPU times: user 2min 38s, sys: 24.2 s, total: 3min 2s
Wall time: 18min 20s


-----
**Analysing NERs across the years**

In [23]:
%%time
df_filt_2020=pd.read_csv('gs://nlp_final_project_kshitijm/00_Data/NLP_FP_Data5_2021_Topics_Sentiments_NER.csv',lineterminator='\n')
df_filt_2021=pd.read_csv('gs://nlp_final_project_kshitijm/00_Data/NLP_FP_Data6_2021v2_Topics_Sentiments_NER.csv',lineterminator='\n')
df_filt_2022=pd.read_csv('gs://nlp_final_project_kshitijm/00_Data/NLP_FP_Data6_2022_Topics_Sentiments_NER.csv', lineterminator='\n')
df_filt_2023=pd.read_csv('gs://nlp_final_project_kshitijm/00_Data/NLP_FP_Data6_2023_Topics_Sentiments_NER.csv', lineterminator='\n')

CPU times: user 20.9 s, sys: 2.85 s, total: 23.7 s
Wall time: 48.2 s


In [24]:
def get_top_ents(df):

    df['ENT_ORG']=df['ENT_ORG'].parallel_apply(ast.literal_eval)
    org_list = list(chain.from_iterable(df['ENT_ORG']))
    counter_org = Counter(org_list)
    top_20_org = counter_org.most_common(20)
    
    df['ENT_PROD']=df['ENT_PROD'].parallel_apply(ast.literal_eval)
    prod_list = list(chain.from_iterable(df['ENT_PROD']))
    counter_prod = Counter(prod_list)
    top_20_prod = counter_prod.most_common(20)
    
    df['ENT_PER']=df['ENT_PER'].parallel_apply(ast.literal_eval)
    per_list = list(chain.from_iterable(df['ENT_PER']))
    counter_per = Counter(per_list)
    top_20_per = counter_per.most_common(20)
    
    df['ENT_NORP']=df['ENT_NORP'].parallel_apply(ast.literal_eval)
    norm_list = list(chain.from_iterable(df['ENT_NORP']))
    counter_norm = Counter(norm_list)
    top_20_norm = counter_norm.most_common(20)
    
    return top_20_org, top_20_prod, top_20_per, top_20_norm


In [ ]:
top_20_org_2020, top_20_prod_2020, top_20_per_2020, top_20_norm_2020 = get_top_ents(df_filt_2020)
top_20_org_2021, top_20_prod_2021, top_20_per_2021, top_20_norm_2021 = get_top_ents(df_filt_2021)
top_20_org_2022, top_20_prod_2022, top_20_per_2022, top_20_norm_2022 = get_top_ents(df_filt_2022)
top_20_org_2023, top_20_prod_2023, top_20_per_2023, top_20_norm_2023 = get_top_ents(df_filt_2023)

## Part 2: Yearly Results

In [35]:
ORG_trend=pd.DataFrame({'2020':top_20_org_2020,'2021':top_20_org_2021,'2022':top_20_org_2022, '2023':top_20_org_2023})
ORG_trend

,2020,2021,2022,2023
0,"(AI, 37756)","(AI, 50705)","(AI, 71025)","(ChatGPT, 155292)"
1,"(V19, 28787)","(V19, 21160)","(Gray Media Group, 54282)","(AI, 84010)"
2,"(Google, 20518)","(Gray Media Group, 19507)","(Station20022022 Gray Television, 23401)","(Microsoft, 53121)"
3,"(Microsoft, 14155)","(Google, 19109)","(Google, 19290)","(Google, 50085)"
4,"(IBM, 13379)","(Microsoft, 14261)","(ML, 12760)","(OpenAI, 29117)"
5,"(Artificial Intelligence, 12495)","(Artificial Intelligence, 13384)","(V19, 9213)","(Gray Media Group, 28657)"
6,"(GR, 11260)","(IBM, 13200)","(Microsoft, 7737)","(Bard, 18375)"
7,"(Facebook, 9600)","(Facebook, 10079)","(ChatGPT, 7456)","(Station20022023 Gray Television, 12870)"
8,"(Artificial Intelligence AI, 7383)","(GR, 9837)","(Facebook, 7436)","(Microsofts, 10510)"
9,"(Amazon, 6893)","(Station20022021 Gray Television, 7721)","(IBM, 7204)","(Googles, 10029)"


In [37]:
PROD_trend=pd.DataFrame({'2020':top_20_prod_2020,'2021':top_20_prod_2021,'2022':top_20_prod_2022, '2023':top_20_prod_2023})
PROD_trend

,2020,2021,2022,2023
0,"(AI, 70971)","(AI, 109573)","(AI, 167203)","(AI, 202427)"
1,"(V19, 2601)","(V19, 2151)","(AIdriven, 3785)","(Bing, 8028)"
2,"(AIdriven, 1194)","(AIdriven, 1929)","(UsMeet, 2337)","(Android, 3517)"
3,"(Twitter YouTube, 1113)","(Twitter YouTube, 1467)","(HPC, 1486)","(AIdriven, 3465)"
4,"(JavaScript, 984)","(JavaScript, 1170)","(JavaScript, 1223)","(Windows, 1832)"
5,"(SE, 718)","(SE, 1124)","(SE, 1152)","(YouTube, 1813)"
6,"(Galaxy, 669)","(HPC, 983)","(Cresta, 1054)","(UsMeet, 1646)"
7,"(YouTube, 572)","(Desktop Alert Notifications .Learn, 795)","(YouTube, 939)","(Windows 11, 1499)"
8,"(TensorFlow, 532)","(CRM, 794)","(Android, 850)","(Excel, 1331)"
9,"(Global AI, 459)","(YouTube, 604)","(ADAS, 822)","(Open AI, 1179)"


In [38]:
PER_trend=pd.DataFrame({'2020':top_20_per_2020,'2021':top_20_per_2021,'2022':top_20_per_2022, '2023':top_20_per_2023})
PER_trend

,2020,2021,2022,2023
0,"(Size, 3979)","(Size, 2994)","(Elon Musk, 2219)","(Bing, 10618)"
1,"(Trump, 3283)","(Biden, 1993)","(Musk, 2192)","(Musk, 8478)"
2,"(Instagram, 1360)","(Closefor, 1908)","(Lemoine, 2044)","(Elon Musk, 7255)"
3,"(Middle EastAfrica, 1277)","(Greta Van, 1880)","(Ai Weiwei, 1766)","(Biden, 4333)"
4,"(Biden, 1046)","(Instagram, 1469)","(Vectorspace AI, 1651)","(Trump, 4095)"
5,"(Richests Richests, 1016)","(Elon Musk, 1220)","(Biden, 1464)","(Sam Altman, 3417)"
6,"(Gebru, 886)","(Jim Thorpe, 1088)","(EnterpriseFor, 1184)","(Bard, 2638)"
7,"(Ai Weiwei, 857)","(Middle EastAfrica, 1087)","(Mark Zuckerberg, 1066)","(Sundar Pichai, 2584)"
8,"(Musk, 808)","(Su Tierra Tiempo, 1085)","(StartedIs Bitcoin, 1006)","(Pichai, 2359)"
9,"(Elon Musk, 779)","(Musk, 1080)","(CoFounder, 961)","(Ernie Bot, 2302)"


In [39]:
NORM_trend=pd.DataFrame({'2020':top_20_norm_2020,'2021':top_20_norm_2021,'2022':top_20_norm_2022, '2023':top_20_norm_2023})
NORM_trend

,2020,2021,2022,2023
0,"(Chinese, 4344)","(Chinese, 5339)","(Chinese, 6519)","(Chinese, 9313)"
1,"(American, 1869)","(European, 2883)","(European, 3094)","(Americans, 5781)"
2,"(European, 1783)","(American, 2178)","(Canadian, 2696)","(American, 5154)"
3,"(British, 1654)","(Canadian, 2097)","(American, 2344)","(Italian, 4704)"
4,"(Canadian, 1486)","(British, 1674)","(British, 1713)","(British, 3858)"
5,"(French, 1077)","(American African American, 1526)","(Americans, 1551)","(Canadian, 3022)"
6,"(Americans, 860)","(French, 1170)","(American African American, 1543)","(European, 2791)"
7,"(German, 847)","(Americans, 1150)","(French, 1500)","(French, 2773)"
8,"(Hindu, 742)","(German, 1082)","(Lunit, 1343)","(German, 2011)"
9,"(Tamil, 541)","(Korean, 1033)","(Ukrainian, 1106)","(Canadians, 1405)"


In [41]:
ORG_trend.to_csv("NER_Trend_ORG.csv",index=False)
PROD_trend.to_csv("PROD_Trend_ORG.csv",index=False)
PER_trend.to_csv("PER_Trend_ORG.csv",index=False)
NORM_trend.to_csv("NORM_Trend_ORG.csv",index=False)

In [ ]:
def get_top_ents_2(df):

    # df['ENT_ORG']=df['ENT_ORG'].parallel_apply(ast.literal_eval)
    org_list = list(chain.from_iterable(df['ENT_ORG']))
    counter_org = Counter(org_list)
    top_20_org = counter_org.most_common(20)
    
    # df['ENT_PROD']=df['ENT_PROD'].parallel_apply(ast.literal_eval)
    prod_list = list(chain.from_iterable(df['ENT_PROD']))
    counter_prod = Counter(prod_list)
    top_20_prod = counter_prod.most_common(20)
    
    # df['ENT_PER']=df['ENT_PER'].parallel_apply(ast.literal_eval)
    per_list = list(chain.from_iterable(df['ENT_PER']))
    counter_per = Counter(per_list)
    top_20_per = counter_per.most_common(20)
    
    # df['ENT_NORP']=df['ENT_NORP'].parallel_apply(ast.literal_eval)
    norm_list = list(chain.from_iterable(df['ENT_NORP']))
    counter_norm = Counter(norm_list)
    top_20_norm = counter_norm.most_common(20)
    
    ent_trend_df=pd.DataFrame({'ORGANIZATION':top_20_org,'PRODUCT':top_20_prod,'PERSON':top_20_per, 'NORM':top_20_norm})
    return ent_trend_df
